In [1]:
import wrds
import pandas as pd

# Establish a connection to the WRDS database
db = wrds.Connection(wrds_username='asherbaraban')

# Adjusted SQL query to fetch data for specific 3-day window
df = db.raw_sql(f"""
    WITH insiders AS (
        SELECT *,
            CONCAT(cusip6, cusip2, cusipx) AS cusipI
        FROM tr_insiders.table1
        WHERE 
            formtype = '4' AND
            cleanse IN ('R', 'H') AND
            trancode IS NOT NULL AND
            acqdisp IS NOT NULL
    ),
    prices AS (
        SELECT 
            cusip,
            datadate,
            prccd,
            trfd,
            ajexdi
        FROM 
            comp_na_daily_all.secd
        WHERE 
            datadate >= (SELECT MIN(trandate) FROM insiders) AND
            datadate <= (SELECT MAX(trandate) + INTERVAL '6 months' + INTERVAL '2 days' FROM insiders)
    )
    SELECT
        i.*,
        c.prccd AS trandate_price,
        c.datadate AS trandate_datadate,
        c.cusip,
        c_next1.ajexdi AS ajexdi_6mo_day1,
        c_next1.trfd AS trfd_6mo_day1,
        c_next1.prccd as price_6mo_day1,
        c_next2.ajexdi AS ajexdi_6mo_day2,
        c_next2.trfd AS trfd_6mo_day2,
        c_next2.prccd as price_6mo_day2,
        c_next3.ajexdi AS ajexdi_6mo_day3,
        c_next3.trfd AS trfd_6mo_day3,
        c_next3.prccd as price_6mo_day3,
        c.ajexdi,
        c.trfd
    FROM
        insiders AS i
    LEFT JOIN
        prices AS c
    ON
        i.cusipI = c.cusip AND
        c.datadate = i.trandate
    LEFT JOIN
        prices AS c_next1
    ON
        i.cusipI = c_next1.cusip AND
        c_next1.datadate = i.trandate + INTERVAL '6 months'
    LEFT JOIN
        prices AS c_next2
    ON
        i.cusipI = c_next2.cusip AND
        c_next2.datadate = i.trandate + INTERVAL '6 months' + INTERVAL '1 day'
    LEFT JOIN
        prices AS c_next3
    ON
        i.cusipI = c_next3.cusip AND
        c_next3.datadate = i.trandate + INTERVAL '6 months' + INTERVAL '2 days'
""")

df.to_csv("raw_pull_compustat.csv", index=False)

Loading library list...
Done


OperationalError: (psycopg2.OperationalError) could not receive data from server: Operation timed out
SSL SYSCALL error: Operation timed out

[SQL: 
    WITH insiders AS (
        SELECT *,
            CONCAT(cusip6, cusip2, cusipx) AS cusipI
        FROM tr_insiders.table1
        WHERE 
            formtype = '4' AND
            cleanse IN ('R', 'H') AND
            trancode IS NOT NULL AND
            acqdisp IS NOT NULL
    ),
    prices AS (
        SELECT 
            cusip,
            datadate,
            prccd,
            trfd,
            ajexdi
        FROM 
            comp_na_daily_all.secd
        WHERE 
            datadate >= (SELECT MIN(trandate) FROM insiders) AND
            datadate <= (SELECT MAX(trandate) + INTERVAL '6 months' + INTERVAL '2 days' FROM insiders)
    )
    SELECT
        i.*,
        c.prccd AS trandate_price,
        c.datadate AS trandate_datadate,
        c.cusip,
        c_next1.ajexdi AS ajexdi_6mo_day1,
        c_next1.trfd AS trfd_6mo_day1,

In [ ]:
df[['trandate_datadate', 'cusip6', 'cusip2', 'cusipx',
    'cusip', 'trfd', 'dcn', 'seqnum', 'trfd_6mo_day1',
    'ajexdi_6mo_day1', 'ajexdi_6mo_day2']]







In [2]:
import inspect


# Inspect the function
signature = inspect.signature(db.raw_sql)

# Extract parameters
parameters = signature.parameters

# Print parameter details
for param in parameters.values():
    print(f"Name: {param.name}")
    print(f"Default: {param.default}")
    print(f"Kind: {param.kind}")
    print(f"Annotation: {param.annotation}")
    print()

Name: sql
Default: <class 'inspect._empty'>
Kind: 1
Annotation: <class 'inspect._empty'>

Name: coerce_float
Default: True
Kind: 1
Annotation: <class 'inspect._empty'>

Name: date_cols
Default: None
Kind: 1
Annotation: <class 'inspect._empty'>

Name: index_col
Default: None
Kind: 1
Annotation: <class 'inspect._empty'>

Name: params
Default: None
Kind: 1
Annotation: <class 'inspect._empty'>

Name: chunksize
Default: 500000
Kind: 1
Annotation: <class 'inspect._empty'>

Name: return_iter
Default: False
Kind: 1
Annotation: <class 'inspect._empty'>



In [2]:
import wrds
import pandas as pd

# Establish a connection to the WRDS database
db = wrds.Connection(wrds_username='asherbaraban')

# # Fetch insiders data
# insiders_df = db.raw_sql(f"""
#     SELECT *,
#         CONCAT(cusip6, cusip2, cusipx) AS cusipI
#     FROM tr_insiders.table1
#     WHERE 
#         formtype = '4' AND
#         cleanse IN ('R', 'H') AND
#         trancode IS NOT NULL AND
#         acqdisp IS NOT NULL
# """)

# insiders_df

Loading library list...
Done


In [5]:
x = db.get_table(library='tr_insiders', table='table1', obs=10)

In [6]:
for i in x.columns:
    print(i)

fdate
cdate
dcn
seqnum
formtype
personid
owner
secid
ticker
cusip6
cusip2
cusipx
cname
rolecode1
rolecode2
rolecode3
rolecode4
trancode
acqdisp
trandate
tprice
ownership
shares
sharesheld
amend
secdate
sigdate
maintdate
cleanse
trancode_ar
acqdisp_ar
tprice_ar
trandate_ar
optionsell
sectitle
shares_adj
sharesheld_adj
tprice_adj
address1
address2
city
state
zipcode
country
phone
cnum
sector
industry


In [7]:
x

,fdate,cdate,dcn,seqnum,formtype,personid,owner,secid,ticker,cusip6,...,address1,address2,city,state,zipcode,country,phone,cnum,sector,industry
0,2016-05-17,2000-03-23,00011111,1.0,5,12223429.0,BARTELS JUERGEN,8744.0,HOT,85590A,...,14665 MIDWAY ROAD,None,ADDISON,TX,75244,None,None,S721390000,04,02
1,2014-03-07,2000-11-07,001249601,1.0,4,12086811.0,MAY EARLE C,8051.0,None,94767P,...,696 MCVEY AVE,PO BOX 1629,LAKE OSWEGO,OR,97034,None,None,W247877853,01,02
2,2014-03-07,2000-11-07,001249601,2.0,4,12086811.0,MAY EARLE C,8051.0,None,94767P,...,696 MCVEY AVE,PO BOX 1629,LAKE OSWEGO,OR,97034,None,None,W247877853,01,02
3,2014-03-07,2000-11-07,001249601,3.0,4,12086811.0,MAY EARLE C,8051.0,None,94767P,...,696 MCVEY AVE,PO BOX 1629,LAKE OSWEGO,OR,97034,None,None,W247877853,01,02
4,2007-08-17,2000-05-30,0013116598,1.0,3,13116598.0,COOPER HELEN E,22737.0,None,03061Q,...,None,None,None,None,None,None,None,A622351725,None,None
5,2007-08-17,2000-05-30,0013116603,1.0,3,13116603.0,MACHADO CRAIG D,22737.0,None,03061Q,...,None,None,None,None,None,None,None,A622351725,None,None
6,2007-08-17,2000-05-30,0013116606,1.0,3,13116606.0,THOMAS CHRISTOPHER,22737.0,None,03061Q,...,None,None,None,None,None,None,None,A622351725,None,None
7,2007-08-17,2001-01-08,001502679,1.0,4,12314118.0,IVY MADIE,19327.0,ESREF,G31215,...,1330 AVENUE OF THE AMERICAS,None,NEW YORK,NY,10019,None,None,E822975000,01,05
8,2007-08-17,2001-01-08,001502679,2.0,4,12314118.0,IVY MADIE,19327.0,ESREF,G31215,...,1330 AVENUE OF THE AMERICAS,None,NEW YORK,NY,10019,None,None,E822975000,01,05
9,2007-08-17,2001-01-08,001502679,3.0,4,12314118.0,IVY MADIE,19327.0,ESREF,G31215,...,1330 AVENUE OF THE AMERICAS,None,NEW YORK,NY,10019,None,None,E822975000,01,05
